In [1]:
import os

try:
    import subprocess
    import sys

    from google.colab import drive

    subprocess.run(["pip", "install", "torchmetrics", "optuna"])
    base_dir = "/content/drive/MyDrive/Colab_Notebooks/Crack_Detection"
    drive.mount("/content/drive")
    sys.path.append(os.path.join(base_dir, "semantic-segmentation"))
    LOCAL = False
except ImportError:
    base_dir = "."
    LOCAL = True

import matplotlib.pyplot as plt
import numpy as np
import optuna
import torch
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import find_boundaries
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall
from torchmetrics.clustering import RandScore

from train import (
    DataLoader,
    ImageMaskTransform,
    SegmentationDataset,
    UNet,
    cross_entropy_weighted,
    init_data_loaders,
    init_datasets,
    tune_hyperparams,
    train_fixed_hyperparams,
)


%load_ext tensorboard

In [ ]:
%tensorboard --logdir {os.path.join(base_dir, "runs")}

In [ ]:
tune_hyperparams(base_dir, LOCAL)

In [2]:
params = {
    "batch_size": 8,
    "brightness_prob": 0.1385,
    "dropout_p": 0.5329,
    "elastic_prob": 0.0157,
    "flip_prob": 0.0427,
    "lr": 0.000363,
    "lr_cooldown": 1,
    "lr_factor": 0.0693,
    "lr_patience": 18,
    "min_lr": 1.002e-08,
    "rotate_prob": 0.472,
    "translate_prob": 0.1538,
    "use_adam": True,
    "use_cosine_scheduler": False,
    "vanilla_loss": True,
}
num_epochs = 500
min_save_epoch = 30
early_stop_patience = 50
train_fixed_hyperparams(
    base_dir, LOCAL, params, num_epochs, min_save_epoch, early_stop_patience
)

Epoch 1/500, Loss/train: 0.0289, RandError/train: 0.3773, PixelError/train: 0.2523, Recall/train: 1.0000, Precision/train: 0.7477, Loss/val: 0.1128, VanillaLoss/val: 0.1128, RandError/val: 0.3202, PixelError/val: 0.2002, Recall/val: 1.0000, Precision/val: 0.7998
Epoch 2/500, Loss/train: 0.0284, RandError/train: 0.3289, PixelError/train: 0.2075, Recall/train: 1.0000, Precision/train: 0.7925, Loss/val: 0.1102, VanillaLoss/val: 0.1102, RandError/val: 0.3202, PixelError/val: 0.2002, Recall/val: 1.0000, Precision/val: 0.7998
Epoch 3/500, Loss/train: 0.0278, RandError/train: 0.3447, PixelError/train: 0.2213, Recall/train: 1.0000, Precision/train: 0.7787, Loss/val: 0.1066, VanillaLoss/val: 0.1066, RandError/val: 0.3202, PixelError/val: 0.2002, Recall/val: 1.0000, Precision/val: 0.7998
Epoch 4/500, Loss/train: 0.0261, RandError/train: 0.3707, PixelError/train: 0.2457, Recall/train: 1.0000, Precision/train: 0.7543, Loss/val: 0.1018, VanillaLoss/val: 0.1018, RandError/val: 0.3202, PixelError/val

KeyboardInterrupt: 

In [7]:
import pickle
import optuna.visualization as vis

# study_name = "study-0506-190624"
study_name = "study-0502-175711"
# study_name = "study-0428"
storage = f"sqlite:///Data/{study_name}/Data/seg-study.db"
studies = optuna.study.get_all_study_summaries(storage=storage)
print("Num of studies:", len(studies))
study = studies[-1]
print(study.study_name)

Num of studies: 14
study-0502-175709


In [8]:
try:
    with open(f"Data/{study_name}/Data/studies/{study.study_name}.pkl", "rb") as f:
        study_dict = pickle.load(f)

    study = optuna.load_study(
        study_name=study.study_name,
        storage=storage,
        sampler=study_dict["sampler"],
        pruner=study_dict["pruner"],
    )
except FileNotFoundError:
    study = optuna.load_study(study_name=study.study_name, storage=storage)

In [9]:
study.best_value, len(study.trials), study.pruner, study.best_params

(0.13715654611587524,
 47,
 {'flip_prob': 0.042747388239577966,
  'rotate_prob': 0.4719902190284881,
  'elastic_prob': 0.01567998717280196,
  'translate_prob': 0.15377448395168106,
  'brightness_prob': 0.138486416623626,
  'batch_size': 8,
  'dropout_p': 0.5328890961652364,
  'vanilla_loss': True,
  'use_adam': True,
  'lr': 0.00036296166101104416,
  'use_cosine_scheduler': False,
  'min_lr': 1.0017766083008169e-08,
  'lr_patience': 18,
  'lr_cooldown': 1,
  'lr_factor': 0.06925485872989952})

In [29]:
study.best_value, len(study.trials), study.pruner, study.best_params

(0.14338421821594238,
 100,
 {'flip_prob': 0.04459255293028834,
  'rotate_prob': 0.40379419104333913,
  'elastic_prob': 0.014062767704262479,
  'translate_prob': 0.33608821897374763,
  'brightness_prob': 0.23828063471331679,
  'batch_size': 9,
  'dropout_p': 0.2887745942859943,
  'vanilla_loss': True,
  'use_adam': True,
  'lr': 0.0003909349124991186,
  'use_cosine_scheduler': False,
  'min_lr': 6.945255506527192e-08,
  'lr_patience': 24,
  'lr_cooldown': 1,
  'lr_factor': 0.2878405900553068})

In [10]:
for trial in study.trials:
    params = trial.params
    if not params["vanilla_loss"]:
        v = trial.value
        if v and v < 0.25:
            print(f"{v:0.3f}", trial.number, trial.params, trial.state)

In [14]:
optuna.importance.get_param_importances(study)

{'use_adam': np.float64(0.3977645095745252),
 'vanilla_loss': np.float64(0.1406793931554693),
 'elastic_prob': np.float64(0.08746186350862907),
 'batch_size': np.float64(0.08671901945241697),
 'use_cosine_scheduler': np.float64(0.08402969690245228),
 'brightness_prob': np.float64(0.06316926781526236),
 'translate_prob': np.float64(0.043377337361138406),
 'rotate_prob': np.float64(0.031254978146683376),
 'flip_prob': np.float64(0.025227132797285463),
 'min_lr': np.float64(0.021256423150298954),
 'dropout_p': np.float64(0.0190603781358387)}

In [ ]:
vis.plot_param_importances(study)

In [ ]:
(train_image_dir, train_images, train_mask_dir, val_images, val_percent) = (
    init_datasets(base_dir)
)

flip_prob = 0.1
rotate_prob = 0.1
elastic_prob = 0.11
translate_prob = 0.1
brightness_prob = 0.1
batch_size = 1 if LOCAL else 9
train_dataloader, val_dataloader = init_data_loaders(
    batch_size,
    brightness_prob,
    elastic_prob,
    flip_prob,
    rotate_prob,
    train_image_dir,
    train_images,
    train_mask_dir,
    translate_prob,
    val_images,
)

In [ ]:
for images, masks in train_dataloader:
    image = images[0].permute(1, 2, 0).cpu().numpy()
    mask = masks[0].cpu().numpy().squeeze()
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(image, cmap="gray")
    axs[1].imshow(mask, cmap="gray")
    assert image.shape[:2] == (572, 572)
    assert mask.shape == (388, 388)
    plt.show()

In [ ]:
loss_w0, loss_sigma = 5, 5
loss_w1 = 1.0
dropout_p = 0.2
vanilla_loss = True

test_image_dir = os.path.join(base_dir, "isbi_2012_challenge/test/imgs")
test_mask_dir = os.path.join(base_dir, "isbi_2012_challenge/test/labels")

test_images = os.listdir(test_image_dir)
test_transforms = ImageMaskTransform(train=False)
test_dataset = SegmentationDataset(
    test_image_dir, test_mask_dir, test_images, transform=test_transforms
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

pretrained_weights_path = "checkpoints/C1.pth"
model = UNet(dropout_p=dropout_p)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.load_state_dict(torch.load(pretrained_weights_path, map_location=device))
model.eval()

accuracy_metric_test = BinaryAccuracy().to(device)
precision_metric_test = BinaryPrecision().to(device)
recall_metric_test = BinaryRecall().to(device)
rand_score_metric_test = RandScore().to(device)
test_loss = 0.0
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        labels = labels.squeeze(1)
        loss = cross_entropy_weighted(
            outputs,
            labels,
            device,
            loss_w0,
            loss_sigma,
            loss_w1,
            vanilla=vanilla_loss,
        )
        test_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        accuracy_metric_test.update(preds, labels)
        recall_metric_test.update(preds, labels)
        precision_metric_test.update(preds, labels)
        rand_score_metric_test.update(preds.view(-1), labels.view(-1))

print(
    f"(Test) Loss: {test_loss / len(test_dataloader):.4f}, "
    f"Rand error: {1 - rand_score_metric_test.compute():.4f} "
    f"Pixel Error: {1 - accuracy_metric_test.compute():.4f} "
    f"Recall: {recall_metric_test.compute():.4f} "
    f"Precision: {precision_metric_test.compute():.4f}"
)

In [ ]:
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        image = images[0].permute(1, 2, 0).cpu().numpy()
        mask = labels[0].cpu().numpy().squeeze()

        predicted_mask = torch.argmax(outputs[0], dim=0).cpu().numpy()

        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        axs[0].imshow(image, cmap="gray")
        axs[0].set_title("Image")
        axs[0].axis("off")

        axs[1].imshow(mask, cmap="gray")
        axs[1].set_title("Ground Truth Mask")
        axs[1].axis("off")

        axs[2].imshow(predicted_mask, cmap="gray")
        axs[2].set_title("Predicted Mask")
        axs[2].axis("off")

plt.show()

In [ ]:
W0, SIGMA = 5, 5
for images, labels in train_dataloader:
    borders = find_boundaries(labels)
    dist = distance_transform_edt(~borders)
    w = W0 * np.exp(-2 * dist**2 / SIGMA**2)

    labels_bincount = torch.bincount(labels.flatten())
    w_class = labels_bincount.sum() / labels_bincount
    print(w_class, labels_bincount.sum())

    class_map = w_class[labels]
    print(class_map)

    w_final = class_map.numpy() + w

    fig, axs = plt.subplots(1, 4, figsize=(19, 6))
    axs[0].imshow(labels.squeeze(), cmap="gray")
    axs[1].imshow(~borders.squeeze(), cmap="gray")
    axs[2].imshow(w.squeeze(), cmap="coolwarm")
    axs[3].imshow(w_final.squeeze(), cmap="coolwarm")
    plt.show()
    plt.hist(w.flatten(), bins=10)
    plt.show()
    plt.hist(w_final.flatten(), bins=10)
    break